# Imports

In [1]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
#!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile
%matplotlib inline

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [3]:
!pip3 install librosa

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [4]:
import spacy
import functools
import numpy as np
import pandas as pd
import os
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
from keras.layers import Dense, Dropout, Input, Flatten
from keras import Model
import pandas as pd
import numpy as np
import os
import scipy.io.wavfile as wavfile
from matplotlib import pyplot as plt
from IPython import display
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import soundfile as sf
from wav2vec2 import Wav2Vec2Processor
import librosa

import random

# Audio Pre-Processing

In [5]:
REQUIRED_SAMPLE_RATE=16000

In [6]:
def read_wav_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".wav")]
  return audio

In [7]:
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

In [8]:
def add_padding(wavform, length_in_seconds=15.375):
    input_len = int(REQUIRED_SAMPLE_RATE * length_in_seconds)
    wavform = wavform[:input_len]
    zero_padding = tf.zeros(
        [input_len] - tf.shape(wavform),
        dtype=tf.float32
    )
    wavform = tf.cast(wavform, dtype=tf.float32)
    equal_length = tf.concat([wavform, zero_padding], 0)
    return equal_length

In [9]:
def convert_audio_path_to_vec(audio_list: list):
    res = audio_list.copy()
    res = list(
        map(
            lambda path: read_wav_file(path),
            res
        )
    )
    res = list(
        map(
            lambda wav: preprocess_speech(wav),
            res
        )
    )
    res = list(
        map(
            lambda vec: add_padding(wavform=vec),
            res
        )
    )
    res = np.array(res)
    return res

# Read Data

In [10]:
df = pd.read_csv('../Data/textVideoClassification.tsv',sep='\t')

x_audio_val, x_text_val = {1:[],2:[],3:[],4:[],5:[]}, {1:[],2:[],3:[],4:[],5:[]}
y_val = {1:[],2:[],3:[],4:[],5:[]}

for i, row in df.iterrows():
    text = row['text']
    audio_path = os.path.join('../Data','Audio',row['session'],'{}-{}.wav'.format(row['audio'],row['label']))
    label =  np.array(
        [float(row['label']=='fru'),float(row['label']=='ang'),float(row['label']=='sad'),float(row['label']=='hap' or row['label']=='exc')]
    )
    ses_num = int(row['session'][-1])
    x_text_val[ses_num].append(text)
    x_audio_val[ses_num].append(audio_path)
    y_val[ses_num].append(label)

for ses_num in x_audio_val:
    x_text_val[ses_num] = np.array(x_text_val[ses_num])
    y_val[ses_num] = np.array(y_val[ses_num])
    x_audio_val[ses_num] = convert_audio_path_to_vec(x_audio_val[ses_num])

# Corrupting Text Methods

In [11]:
def is_adjective(token):
    return token.pos_ == 'ADJ'

In [12]:
def is_noun(token):
    return token.pos_ == 'NOUN'

In [13]:
def mask(sentence: str, to_mask) -> str:
    """
    convert a text instance to be partially masked according to a given predicate
    :param sentence: text utterance
    :param to_mask: predicate to determine which tokens should be masked
    :return: the input sentence with [MASK] tokens instead of the original selected tokens
    """
    nlp = spacy.load('en_core_web_sm')
    token_list = nlp(sentence)
    token_list = list(
        map(
            lambda token: '[Mask]' if to_mask(token) else token.text,
            token_list
        )
    )
    res = functools.reduce(
        lambda a, b: a + ' ' + b,
        token_list
    )
    return res

In [14]:
def count_by_predicate(sentence: str, to_mask) -> int:
    '''
    count how many tokens appears in a text instance according to a given predicate
    :param sentence: the text instance
    :param to_mask: predicate to select which tokens we want to count
    '''
    nlp = spacy.load('en_core_web_sm')
    token_list = nlp(sentence)
    count = 0
    for token in token_list:
        if(to_mask(token)):
            count += 1
    return count

In [15]:
def mask_by_count_tokens(sentence: str, x):
    '''
    mask X tokens randomly in text instance
    :param sentence: text instance
    :param x: determine how many tokens should be masked
    :return: the text instance after masking tokens
    '''
    nlp = spacy.load('en_core_web_sm')
    token_list = nlp(sentence)
    selected = random.sample(range(len(token_list)), k=x)
    res = ''
    for i in range(len(token_list)):
        if i in selected:
            res += '[MASK] '
        else:
            res += token_list[i].text + ' '
    return res

In [16]:
def mask_random_tokens(data,predicate):
    '''
    mask randomly tokens over a collection of text utterances
    :param data: the text data collection
    :param predicate: determines how many tokens to mask
    :return: the collection with the masked text
    '''
    count_list = []
    for i in range(len(data)):
        count_list.append(count_by_predicate(str(data[i]), predicate))
    res = []
    for i in range(len(data)):
        res.append(mask_by_count_tokens(str(data[i]), count_list[i]))

    return np.array(res)

In [17]:
def mask_adj(data):
    '''
    mask all adjectives over the text instances collection
    :param data: the text instances collection
    :return: the data after masking all the adjectives
    '''
    tmp = data.copy()
    for i in range(len(tmp)):
        tmp[i] = mask(str(tmp[i]),is_adjective)
    return np.array(tmp)

In [18]:
def mask_nouns(data):
    '''
    mask all nouns over the text instances collection
    :param data: the text instances collection
    :return: the data after masking all the adjectives
    '''
    tmp = data.copy()
    for i in range(len(tmp)):
        tmp[i] = mask(str(tmp[i]),is_noun)
    return np.array(tmp)

# Corrupting Audio Methods

In [19]:
from random import randint
def change_pitch(data, sample_rate=16000, random_pitch = True):
    if random_pitch:
        pitch_factor = randint(-1,1)
    return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)

In [20]:
def get_audio_with_changed_pitch(data):
    return np.array(
        list(
            map(
                lambda audio: change_pitch(audio),
                data
            )
        )
    )

# Test Models

## Mask All Adj.

In [21]:
for ses_num in range(1,6):
    print('---------------Ses0{}------------------'.format(ses_num))
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('masking adj.')
    text = mask_adj(x_text_val[ses_num])
    model.evaluate((x_audio_val[ses_num],text),y_val[ses_num])

---------------Ses01------------------
baseline
31/31 [==============================] - 29s 780ms/step - loss: 0.9357 - accuracy: 0.7533
masking adj.
31/31 [==============================] - 23s 750ms/step - loss: 1.0937 - accuracy: 0.7054
---------------Ses02------------------
baseline
31/31 [==============================] - 26s 790ms/step - loss: 0.6236 - accuracy: 0.7779
masking adj.
31/31 [==============================] - 23s 753ms/step - loss: 0.7342 - accuracy: 0.7383
---------------Ses03------------------
baseline
38/38 [==============================] - 32s 786ms/step - loss: 0.7348 - accuracy: 0.7296
masking adj.
38/38 [==============================] - 29s 758ms/step - loss: 0.7968 - accuracy: 0.6966
---------------Ses04------------------
baseline
40/40 [==============================] - 32s 754ms/step - loss: 0.8540 - accuracy: 0.7321
masking adj.
40/40 [==============================] - 30s 746ms/step - loss: 1.0408 - accuracy: 0.6667
---------------Ses05----------------

## Mask Randomly (#Adj.)

In [22]:
for ses_num in range(1,6):
    print('---------------Ses0{}------------------'.format(ses_num))
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('masking random tokens (#Adj.)')
        text = mask_random_tokens(x_text_val[ses_num], is_adjective)
        model.evaluate((x_audio_val[ses_num],text),y_val[ses_num])

---------------Ses01------------------
baseline
31/31 [==============================] - 25s 752ms/step - loss: 0.9357 - accuracy: 0.7533
-----experiment 1-----
masking random tokens (#Adj.)
31/31 [==============================] - 23s 750ms/step - loss: 1.0388 - accuracy: 0.7217
-----experiment 2-----
masking random tokens (#Adj.)
31/31 [==============================] - 23s 750ms/step - loss: 1.0420 - accuracy: 0.7238
-----experiment 3-----
masking random tokens (#Adj.)
31/31 [==============================] - 23s 750ms/step - loss: 1.0707 - accuracy: 0.7187
---------------Ses02------------------
baseline
31/31 [==============================] - 26s 755ms/step - loss: 0.6236 - accuracy: 0.7779
-----experiment 1-----
masking random tokens (#Adj.)
31/31 [==============================] - 23s 751ms/step - loss: 0.6673 - accuracy: 0.7647
-----experiment 2-----
masking random tokens (#Adj.)
31/31 [==============================] - 23s 753ms/step - loss: 0.6798 - accuracy: 0.7606
-----expe

## Mask All Nouns

In [23]:
for ses_num in range(1,6):
    print('---------------Ses0{}------------------'.format(ses_num))
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('masking nouns')
    text = mask_nouns(x_text_val[ses_num])
    model.evaluate((x_audio_val[ses_num],text),y_val[ses_num])

---------------Ses01------------------
baseline
31/31 [==============================] - 26s 753ms/step - loss: 0.9357 - accuracy: 0.7533
masking nouns
31/31 [==============================] - 23s 750ms/step - loss: 1.0735 - accuracy: 0.6932
---------------Ses02------------------
baseline
31/31 [==============================] - 26s 757ms/step - loss: 0.6236 - accuracy: 0.7779
masking nouns
31/31 [==============================] - 23s 753ms/step - loss: 0.7284 - accuracy: 0.7394
---------------Ses03------------------
baseline
38/38 [==============================] - 31s 762ms/step - loss: 0.7348 - accuracy: 0.7296
masking nouns
38/38 [==============================] - 29s 760ms/step - loss: 0.8485 - accuracy: 0.6636
---------------Ses04------------------
baseline
40/40 [==============================] - 32s 747ms/step - loss: 0.8540 - accuracy: 0.7321
masking nouns
40/40 [==============================] - 30s 744ms/step - loss: 1.1169 - accuracy: 0.6252
---------------Ses05------------

## Mask Randomly Tokens (#Nouns)

In [24]:
for ses_num in range(1,6):
    print('---------------Ses0{}------------------'.format(ses_num))
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('masking random tokens (#Nouns)')
        text = mask_random_tokens(x_text_val[ses_num], is_noun)
        model.evaluate((x_audio_val[ses_num],text),y_val[ses_num])

---------------Ses01------------------
baseline
31/31 [==============================] - 26s 753ms/step - loss: 0.9357 - accuracy: 0.7533
-----experiment 1-----
masking random tokens (#Nouns)
31/31 [==============================] - 23s 750ms/step - loss: 1.0427 - accuracy: 0.7166
-----experiment 2-----
masking random tokens (#Nouns)
31/31 [==============================] - 23s 752ms/step - loss: 0.9882 - accuracy: 0.7146
-----experiment 3-----
masking random tokens (#Nouns)
31/31 [==============================] - 23s 750ms/step - loss: 1.0720 - accuracy: 0.7125
---------------Ses02------------------
baseline
31/31 [==============================] - 26s 755ms/step - loss: 0.6236 - accuracy: 0.7779
-----experiment 1-----
masking random tokens (#Nouns)
31/31 [==============================] - 23s 752ms/step - loss: 0.6754 - accuracy: 0.7566
-----experiment 2-----
masking random tokens (#Nouns)
31/31 [==============================] - 23s 753ms/step - loss: 0.6733 - accuracy: 0.7606
----

## Corrupt Pitch Randomly

In [25]:
for ses_num in range(1,6):
    print('---------------Ses0{}------------------'.format(ses_num))
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('changing audio pitch')
        audio = get_audio_with_changed_pitch(x_audio_val[ses_num])
        model.evaluate((audio,x_text_val[ses_num]),y_val[ses_num])


---------------Ses01------------------
baseline
31/31 [==============================] - 26s 754ms/step - loss: 0.9357 - accuracy: 0.7533
-----experiment 1-----
changing audio pitch


<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: F

31/31 [==============================] - 23s 752ms/step - loss: 1.0833 - accuracy: 0.7288
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 753ms/step - loss: 1.0611 - accuracy: 0.7278
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 751ms/step - loss: 1.0586 - accuracy: 0.7319
---------------Ses02------------------
baseline
31/31 [==============================] - 25s 756ms/step - loss: 0.6236 - accuracy: 0.7779
-----experiment 1-----
changing audio pitch
31/31 [==============================] - 23s 755ms/step - loss: 0.6707 - accuracy: 0.7708
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 754ms/step - loss: 0.6729 - accuracy: 0.7738
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 755ms/step - loss: 0.6787 - accuracy: 0.7698
---------------Ses03------------------
baseline
38/38 [==============================] - 31s 761ms/ste

## Mask Randomly Tokens (#Adj.) + Corrupted Pitch

In [26]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('changing audio pitch')
        audio = get_audio_with_changed_pitch(x_audio_val[ses_num])
        print('masking rnadom tokens in text')
        text = mask_random_tokens(x_text_val[ses_num],is_adjective)
        model.evaluate((audio,text),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 26s 755ms/step - loss: 0.9357 - accuracy: 0.7533
-----experiment 1-----
changing audio pitch


<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=-1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)


masking rnadom tokens in text
31/31 [==============================] - 23s 750ms/step - loss: 1.1816 - accuracy: 0.7044
-----experiment 2-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 750ms/step - loss: 1.1828 - accuracy: 0.6983
-----experiment 3-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 751ms/step - loss: 1.1915 - accuracy: 0.6983
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 26s 755ms/step - loss: 0.6236 - accuracy: 0.7779
-----experiment 1-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 753ms/step - loss: 0.7318 - accuracy: 0.7576
-----experiment 2-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 753ms/step - loss: 0.7653 - accuracy: 0.7373
-----experiment 3-----
changing audio pitch
maski

## Mask Randomly Tokens (#Nouns) + Corrupted Pitch

In [27]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('changing audio pitch')
        audio = get_audio_with_changed_pitch(x_audio_val[ses_num])
        print('masking rnadom tokens in text')
        text = mask_random_tokens(x_text_val[ses_num],is_noun)
        model.evaluate((audio,text),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 25s 752ms/step - loss: 0.9357 - accuracy: 0.7533
-----experiment 1-----
changing audio pitch


<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=-1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)


masking rnadom tokens in text
31/31 [==============================] - 23s 752ms/step - loss: 1.2180 - accuracy: 0.7044
-----experiment 2-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 751ms/step - loss: 1.2807 - accuracy: 0.6850
-----experiment 3-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 749ms/step - loss: 1.3125 - accuracy: 0.6718
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 26s 756ms/step - loss: 0.6236 - accuracy: 0.7779
-----experiment 1-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 754ms/step - loss: 0.7081 - accuracy: 0.7647
-----experiment 2-----
changing audio pitch
masking rnadom tokens in text
31/31 [==============================] - 23s 755ms/step - loss: 0.7399 - accuracy: 0.7546
-----experiment 3-----
changing audio pitch
maski

## Mask All Adj. + Corrupted Pitch

In [28]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('masking adj.')
    text = mask_adj(x_text_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('changing audio pitch')
        audio = get_audio_with_changed_pitch(x_audio_val[ses_num])
        model.evaluate((audio,text),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 26s 752ms/step - loss: 0.9357 - accuracy: 0.7533
masking adj.
-----experiment 1-----
changing audio pitch


<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=-1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)


31/31 [==============================] - 23s 749ms/step - loss: 1.2876 - accuracy: 0.6881
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 754ms/step - loss: 1.2653 - accuracy: 0.6830
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 753ms/step - loss: 1.2947 - accuracy: 0.6799
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 28s 757ms/step - loss: 0.6236 - accuracy: 0.7779
masking adj.
-----experiment 1-----
changing audio pitch
31/31 [==============================] - 23s 755ms/step - loss: 0.8049 - accuracy: 0.7252
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 756ms/step - loss: 0.8023 - accuracy: 0.7191
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 755ms/step - loss: 0.8156 - accuracy: 0.7231
-------------------testing model for ses03----------

## Mask All Nouns + Corrupted Pitch

In [29]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('masking nouns')
    text = mask_nouns(x_text_val[ses_num])
    for i in range(3):
        print('-----experiment {}-----'.format(i+1))
        print('changing audio pitch')
        audio = get_audio_with_changed_pitch(x_audio_val[ses_num])
        model.evaluate((audio,text),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 26s 754ms/step - loss: 0.9357 - accuracy: 0.7533
masking nouns
-----experiment 1-----
changing audio pitch


<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)
<ipython-input-19-d8935d2556bf>:5: FutureWarning: Pass sr=16000, n_steps=-1 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(data,sample_rate, pitch_factor)


31/31 [==============================] - 23s 749ms/step - loss: 1.2824 - accuracy: 0.6738
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 752ms/step - loss: 1.3143 - accuracy: 0.6687
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 752ms/step - loss: 1.2932 - accuracy: 0.6616
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 26s 757ms/step - loss: 0.6236 - accuracy: 0.7779
masking nouns
-----experiment 1-----
changing audio pitch
31/31 [==============================] - 23s 755ms/step - loss: 0.8002 - accuracy: 0.7211
-----experiment 2-----
changing audio pitch
31/31 [==============================] - 23s 756ms/step - loss: 0.7910 - accuracy: 0.7241
-----experiment 3-----
changing audio pitch
31/31 [==============================] - 23s 756ms/step - loss: 0.7960 - accuracy: 0.7221
-------------------testing model for ses03---------

## Audio = Silence

In [30]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('after muting audio')
    silence = np.zeros(x_audio_val[ses_num].shape)
    model.evaluate((silence,x_text_val[ses_num]),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 26s 753ms/step - loss: 0.9357 - accuracy: 0.7533
after muting audio
31/31 [==============================] - 23s 752ms/step - loss: 1.6298 - accuracy: 0.6045
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 26s 758ms/step - loss: 0.6236 - accuracy: 0.7779
after muting audio
31/31 [==============================] - 23s 756ms/step - loss: 0.9364 - accuracy: 0.7049
-------------------testing model for ses03-------------------
loading model
baseline
38/38 [==============================] - 35s 764ms/step - loss: 0.7348 - accuracy: 0.7296
after muting audio
38/38 [==============================] - 29s 762ms/step - loss: 1.1969 - accuracy: 0.5837
-------------------testing model for ses04-------------------
loading model
baseline
40/40 [==============================] - 33s 751ms/step - loss:

## Empty Text

In [31]:
for ses_num in range(1,6):
    print('-------------------testing model for ses0{}-------------------'.format(ses_num))
    print('loading model')
    model = keras.models.load_model('text+audio-val_Ses0{}-trainable_all'.format(ses_num))
    print('baseline')
    model.evaluate((x_audio_val[ses_num],x_text_val[ses_num]),y_val[ses_num])
    print('empty text')
    empty_text = ["" for i in range(len(x_text_val[ses_num]))]
    empty_text = np.array(empty_text)
    model.evaluate((x_audio_val[ses_num],empty_text),y_val[ses_num])

-------------------testing model for ses01-------------------
loading model
baseline
31/31 [==============================] - 27s 758ms/step - loss: 0.9357 - accuracy: 0.7533
empty text
31/31 [==============================] - 24s 759ms/step - loss: 1.7192 - accuracy: 0.4954
-------------------testing model for ses02-------------------
loading model
baseline
31/31 [==============================] - 26s 761ms/step - loss: 0.6236 - accuracy: 0.7779
empty text
31/31 [==============================] - 24s 762ms/step - loss: 2.0806 - accuracy: 0.3377
-------------------testing model for ses03-------------------
loading model
baseline
38/38 [==============================] - 32s 765ms/step - loss: 0.7348 - accuracy: 0.7296
empty text
38/38 [==============================] - 29s 767ms/step - loss: 1.3446 - accuracy: 0.4213
-------------------testing model for ses04-------------------
loading model
baseline
40/40 [==============================] - 33s 752ms/step - loss: 0.8540 - accuracy: 0.73